#  Pandas tutorial

Pandas (panel data) is the second most useful Python library for data analysis and preparation. It allows to work with tabular data and provides very efficient and easy methods for:
- data selection
- data modification
- data indexing
- merging of data from various sources
- transforming data
- feeding data into `scikit-learn` and other ML-related libraries

In this tutorial we will go through the most useful and common operations performed on pandas.

There are two fundamental data structures that you need to understand:

- `pd.Series`: similar to a list, requires all elements to be of the same type, but provides additional methods and operations
- `pd.DataFrame`: this is the data structure which represents tabular data, each column in a data frame is a `Series` object, in addition each data frame contains a row index and a column index.

Let's dive into coding.

## Manual creation of a data frame

By convention, `pandas` library is imported using the alias `pd`.

The simplest way to create a dataframe is to provide a dictionary of lists. Each key becomes the name of the column, each list becomes the series contained in the column.

In [ ]:
import pandas as pd

df = pd.DataFrame(
{
    'Code': ['PL', 'DE', 'GB', 'CZ'],
    'Name': ['Poland', 'Germany', 'Great Britain', 'Czech Republic'],
    'Population': [38000000, 80000000, 65000000, 10000000]
})

df

Each column is a `pd.Series` object. We can inspect it using either the dot notation, or by referring to the column by its name in brackets.

In [ ]:
df.Population

In [ ]:
df[['Population','Code']]

## Reading data from a file

There are two most common ways of reading text files into `pandas`:
- `pd.read_table`: assumes tab-separated text file
- `pd.read_csv`: assumes comma-separated text file

For the sake of reproducibility we will use public onlie datasets and we will read them directly off the Web. Please take a moment to investigate these datasets:

- [Chipotle orders](https://bit.ly/chiporders)
- [UFO sighting reports](https://bit.ly/uforeports)
- [IMDB movie ratings](https://bit.ly/imdbratings)
- [Drinking by country](https://bit.ly/drinksbycountry)

In [ ]:
orders = pd.read_table('https://bit.ly/chiporders')

orders.head(10)

In [ ]:
ufo = pd.read_csv('https://bit.ly/uforeports')

ufo.head()

Individual series can be concatenated just like strings in Python.

In [ ]:
orders.item_name + ' ' + orders.item_price

## Analyzing a data frame

`pandas` provides simple methods that allow you to investigate the aggregate properties of individual series and the entire data frame.

In [ ]:
movies = pd.read_csv('https://bit.ly/imdbratings')

movies.head()

A simple way to quickly learn the distribution of a feature is to use the `describe()` method

In [ ]:
movies.duration.describe()

The output of the `describe()` method depends on whether the feature is numerical or categorical.

In [ ]:
movies.genre.describe()

One can apply the `describe()` method to the entire dataframe as well.

In [ ]:
movies.describe()

In [ ]:
movies.shape

In [ ]:
movies.columns

In [ ]:
movies.dtypes

For more advanced analysis of `pandas` dataframe we can use the excellent `pandas-profiling` library

In [ ]:
from pandas_profiling import ProfileReport

movies_profile = ProfileReport(df=movies, title="Analysis of the Movies dataframe", explorative=True)
movies_profile

# or simply: movies.profile_report(title="Analysis of the Movies dataframe")

## Renaming columns

In [ ]:
ufo = pd.read_csv('https://bit.ly/uforeports')

ufo.columns

In [ ]:
ufo['Colors Reported']

If a column name contains a space, it can no longer be used with the dot notation. There are many ways a column may be renamed.

In [ ]:
ufo.rename(columns={'Colors Reported': 'Colors_Reported', 'Time': 'Date and tmie'})

In [ ]:
ufo.head()

In [ ]:
col_names = ['city', 'colors_reported', 'shape_reported', 'state', 'time']

pd.read_csv('https://bit.ly/uforeports', names=col_names)

In [ ]:
ufo.columns = col_names + ['non existing column']

ufo.head()

## Dropping rows and columns

An important concept in `pandas` is the concept of an **axis**. An axis is the direction in which an operation is performed. 0-axis refers to an operation that is applied to each row, 1-axis refers to an operation which is applied to all columns.

By default, `pandas` expects the rows to be dropped, so if you want to drop a column, you have to explicitly state `axis=1`.

In [ ]:
drinks = pd.read_csv('https://bit.ly/drinksbycountry')

drinks.head()

In [ ]:
drinks.shape

In [ ]:
drinks.mean()

In [ ]:
drinks.mean(axis=1)

In [ ]:
ufo = pd.read_csv('https://bit.ly/uforeports')

In [ ]:
ufo.head()

In [ ]:
ufo.drop('City', axis=1)

In [ ]:
ufo_backup = ufo.set_index('City')

In [ ]:
ufo_backup.head()

In [ ]:
ufo_backup.drop('Ithaca', axis=0)

In [ ]:
ufo.drop('Time', axis=1)

In [ ]:
ufo.head()

In [ ]:
ufo.drop([1,3,4]).head()

In [ ]:
ufo.index[0:10]

In [ ]:
ufo.drop(ufo.index[0:3]).head()

All these operations do not change the underlying object, but return a modified copy of the object. If you want to perform the operation on the object, you should:
- add `inplace=True`: more efficient and explicit, or
- use assignment: may be a bit slower, but many people prefer this

In [ ]:
ufo.drop('State', axis=1, inplace=True)

ufo.head()

In [ ]:
ufo = ufo.drop([0,1,4])

ufo.head()

## Exercise

1. Read the *Titanic* dataset from https://tinyurl.com/y9p968ys into a dataframe called `titanic`
2. Display first 15 rows of the dataset
3. Rename `PassengerId` to `ID`, `Lname` to `last_name`, and `Name` to `first_name`
4. Remove all rows for which the cabin number is not known

## Sorting data frames

You can sort individual series within a data frame, and you can sort the entire data frame. Sorting can be made permanent.

In [ ]:
movies = pd.read_csv('https://bit.ly/imdbratings')

movies.head()

In [ ]:
movies.duration.sort_index()

In [ ]:
movies.duration.sort_values(ascending=False)

In [ ]:
movies.sort_values('title', ascending=False)

In [ ]:
movies.sort_values(['content_rating', 'duration'])

## Filter rows by a value in a column

The filtering in `pandas` works very similarly to the way we do filtering in `NumPy`. We will start with creating a boolean series based on a condition, and then we will pass this series as an indexer to the data frame.

In [ ]:
movies.genre == 'Horror'

In [ ]:
movies.shape

In [ ]:
movies[movies.genre.isin(['Horror','Thriller','Mystery'])]

If we want to combine several conditions, there are two important things to remember:
- each condition must be in parentheses to help `pandas` establish the order of execution
- instead of `and`, `or` operators we must use `&` and `|`

In [ ]:
movies[ (movies.genre == 'Horror') & (movies.duration > 120)]

We may also negate an index

In [ ]:
movies[~(movies.duration > 120)]

## Exercise

1. Sort the `titanic` dataframe in the decreasing order of the fare price
2. Create a new dataframe `titanic_f` containing only information on female passengers
3. Create a new dataframe `titanic_x` containing only young passenges (age < 18) who did not embark in Cork.

## String methods

`pandas` makes it very easy to use all the string methods on data frame. Just remember to access these methods via the `str` attribute of the series. Method invocations can be easily chained because each method returns a `Series` object.

In [ ]:
orders = pd.read_table('https://bit.ly/chiporders')

orders.head()

In [ ]:
orders.item_name

In [ ]:
orders.item_name.str

In [ ]:
orders.item_name.str.upper().str.lower().str.len()

In [ ]:
orders.item_name.str.replace('Tomato','Pomodoro').str.lower().str.split()

## Exercise

1. Create a list of names of passengers consisting of the title (Mr., Miss., Mrs.) and the last name.
2. Create a list of names of passengers consisting of the first initial and the last name

## Changing data type of a series

All elements in a `Series` object must have the same type. It is possible to cast the entire series to a new type using the builtin `Series.astype()` function. This can be done either during data reading, or after the data frame has been created.

In [ ]:
orders.dtypes

In [ ]:
orders.head()

In [ ]:
orders.order_id.astype(float)

In [ ]:
orders = pd.read_table('https://bit.ly/chiporders', dtype={'quantity': float})

orders.dtypes

If we want to convert `item_price` to a number, we have to first remove the dollar sign from the string representation of the series, and then cast the entire series.

In [ ]:
orders['item_price'] = orders.item_price.str.replace('$','').astype(float)

In [ ]:
orders.dtypes

## Group by

`Pandas` offers a very broad range of methods for advanced data processing. A common operation is to create aggregates of the table based on the grouping of data on a column. This can be easily achieved using a single function call.

In [ ]:
drinks = pd.read_csv('https://bit.ly/drinksbycountry')

drinks.head()

In [ ]:
drinks.mean()

In [ ]:
drinks.groupby('continent')

In [ ]:
drinks.groupby('continent').beer_servings.mean()

In [ ]:
drinks.groupby('continent').std()

We can apply several aggregate functions to a grouped data frame using the `agg()` function

In [ ]:
drinks.groupby('continent').agg(['mean', 'min', 'max', 'count'])

The results of the group by operation can be quickly visualized.

In [ ]:
%matplotlib inline

drinks.groupby('continent').mean().plot(kind='bar')

## Exercise

1. Compute the number of passengers and the average ticket fare based on the port of embarkment
2. Compare the number of female and male passengers who have survived the sinking
3. Compare the mean age of passengers who survived the sinking with the mean age of passengers who have died

## Exploring the data frame

Here we list some useful functions to run after reading the data to get some better understanding of the data.

In [ ]:
movies = pd.read_csv('https://bit.ly/imdbratings')

movies.head()

In [ ]:
movies.star_rating.describe()

In [ ]:
movies.genre.describe()

In [ ]:
movies.genre.value_counts()

In [ ]:
movies.genre.value_counts(normalize=True)

Since the result of the `value_counts()` function is a `Series` object, we can process it further.

In [ ]:
movies.genre.value_counts(normalize=True).plot(kind='bar')

In [ ]:
movies.duration.plot(kind='hist')

In [ ]:
movies.genre.unique()

In [ ]:
genres = movies.genre.unique()

for g in genres:
    df = movies[movies.genre == g]
    ...

A useful method allows you to quickly create pivot tables from series objects.

In [ ]:
genres = movies.genre
ratings = movies.content_rating

In [ ]:
genres

In [ ]:
ratings

In [ ]:
pd.crosstab(genres, ratings)

## Handling missing values

When working with a data frame, we must be careful when the data contains missing values. Two functions are very useful when working with missing values:
- `isnull()`: returns `True` if a value is missing
- `dropna()`: allows to remove rows and/or columns with missing values

In [ ]:
ufo = pd.read_csv('https://bit.ly/uforeports')

ufo.head()

In [ ]:
ufo['Colors Reported'].isnull()

In [ ]:
ufo['Colors Reported'].isnull().sum()

In [ ]:
ufo.isnull().sum()

In [ ]:
ufo.shape

In [ ]:
ufo.dropna(how='all', axis=0, subset=['City', 'Colors Reported']) # all, subset

## Exercise

1. Remove from the `titanic` dataframe records which do not have the age of the passenger registered
2. Compute the number of missing cabin numbers for each class of passengers

## What is an index?

An `Index` is a special type that can be used to access rows and columns. There are three main uses for an index:
- identification of rows/columns
- selection of rows/columns
- alignment of rows

In [ ]:
drinks = pd.read_csv('https://bit.ly/drinksbycountry')

drinks.head()

In [ ]:
drinks.set_index('country', inplace=True)

drinks.head()

Notice that the index of a data frame is inherited by all series.

In [ ]:
drinks.continent.head()

In [ ]:
drinks.continent.value_counts().values

In [ ]:
drinks.continent.value_counts().index

Index is very useful to select both rows and columns. All you need to remember is that `pd.loc` function expects you to provide index entries for rows and columns that you want to retrieve.

In [ ]:
drinks.head()

In [ ]:
drinks.loc['Poland', :]

In [ ]:
drinks.loc['Gabon':'Guyana', 'beer_servings']

In [ ]:
drinks.columns

In [ ]:
drinks.loc[['Poland', 'Germany', 'France'], 'beer_servings':'wine_servings']

Let's create a `Series` object with an index that can be aligned with our `drinks` data frame.

In [ ]:
population = pd.Series([4000000, 38000000, 80000000, 70000000], 
                       index=['Albania', 'Poland', 'Germany', 'France'], 
                       name='population')

population

In [ ]:
drinks.loc[['Albania', 'Poland', 'Germany', 'France', 'Greece']].beer_servings * population

In [ ]:
pd.concat([drinks, population], axis=1)

You can always revert to a default "row number" index and move the index column to the column list.

In [ ]:
drinks.reset_index(inplace=True)

drinks.head()

## Indexing with `loc`, `iloc`, and `ix`

This is quite confusing. Try to remember the following rules:
- `loc` uses row/column indexes (aka labels), the ranges are **inclusive**
- `iloc` uses integer positions on the list of rows and columns, the ranges are **exclusive**
- `ix` was an old way of indexing a dataframe allowing to use both labels and integer positions, it is deprecated

In [ ]:
ufo = pd.read_csv('https://bit.ly/uforeports')

ufo.head()

In [ ]:
ufo.loc[0:3, :]

In [ ]:
ufo.loc[[0,2,4], 'City':'State']

In [ ]:
ufo.iloc[0:3, 1:3]

## Categories and ordered categories

For certain types of columns the data frame can be optimized by switching the type of a column (especially a column used in selection or grouping) into a category type.

In [ ]:
drinks = pd.read_csv('https://bit.ly/drinksbycountry')

drinks.head()

In [ ]:
drinks.info()

In [ ]:
drinks.info(memory_usage='deep')

In [ ]:
drinks.continent.memory_usage(deep=True)

In [ ]:
drinks.continent

In [ ]:
drinks.continent = drinks.continent.astype('category')

drinks.continent.memory_usage(deep=True)

In [ ]:
drinks.continent

You can perform various operations on a category, just remember to access it via `cat` property.

In [ ]:
drinks.continent.cat.codes

In [ ]:
drinks.continent.cat.as_ordered()

Ordered category can be used to allow for logical sorting of rows.

In [ ]:
df = pd.DataFrame({
    'name': ['Mount Everest', 'Kilimanjaro', 'Rysy'],
    'height': ['very high', 'high', 'low']
})

df

In [ ]:
df.sort_values('height')

In [ ]:
from pandas.api.types import CategoricalDtype

heights = CategoricalDtype(categories=['low', 'high', 'very high'], ordered=True)

df['height'] = df.height.astype(heights) 

In [ ]:
df.sort_values('height')

## Exercise

1. Change the index of the `titanic` dataframe to the ticket number
2. Change the `Pclass` attribute into a category. 

## Creating binary variables from categorical columns

Often in data mining we want to binarize categorical features. One of the most common encodings is the dummy encoding, where a feature with `n` values is turned into `n-1` binary columns

In [ ]:
ufo = pd.read_csv('https://bit.ly/uforeports')

ufo.head()

In [ ]:
ufo.State.map({'NY': 'New York', 'NJ': 'New Jersey', 'CO': 'Colorado'})

In [ ]:
pd.get_dummies(ufo.State)

In [ ]:
pd.get_dummies(ufo.State, prefix='state')

In [ ]:
pd.get_dummies(ufo.State, prefix='state').sum(axis=0)

In [ ]:
pd.get_dummies(ufo.State, prefix='state').sum(axis=1)

In [ ]:
df = pd.DataFrame({'gender': ['M', 'F', 'F', 'M', 'F', 'M', 'N', 'N']})
df

In [ ]:
pd.get_dummies(df.gender)

In [ ]:
pd.get_dummies(df.gender, drop_first=True)

## Display options 

The way `pandas` dataframes are displayed insied a notebook can be modified by accessing display options. Two functions are handy for that:
- `get_option()`: check the current setting
- `set_option()`: modify the current setting

Let's change the following settings:
- the number of rows displayed
- the precision of floats
- the maximum width of a column

In [ ]:
movies = pd.read_csv('https://bit.ly/imdbratings')

movies.head()

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 0)
pd.set_option('display.max_colwidth', 25)

In [ ]:
movies

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.precision')
pd.reset_option('display.max_colwidth')

## Applying functions to data frame columns

We can easily apply functions to data frame columns on the fly, or create new columns as the result of applying a function to an existing column. We've seen this behavior before. Two of the most common ways to do it are the `map()` function and the `apply()` function.

In [ ]:
drinks = pd.read_csv('https://bit.ly/drinksbycountry')

drinks.head()

In [ ]:
drinks.country.str.lower()

In [ ]:
drinks.country.map(len)

In [ ]:
pd.concat([drinks.country, drinks.country.map(len)], axis=1)

In [ ]:
drinks.apply(max)

In [ ]:
drinks.apply(max, axis=1)

In [ ]:
drinks.loc[:, 'beer_servings':'wine_servings'].apply(max, axis=1)

In [ ]:
def get_serious_drinkers(total_alcohol: float) -> bool:
    if total_alcohol >= 10.0:
        return True
    else:
        return False

In [ ]:
drinks['heavy_drinkers'] = drinks.total_litres_of_pure_alcohol.apply(get_serious_drinkers)

In [ ]:
drinks.head()

In [ ]:
drinks.total_litres_of_pure_alcohol.apply(lambda x: True if x > 10 else False )

Instead of writing a separate function, many people prefer to use anonymous lambda funciton instead. What you see below is a very common pattern for `pandas` processing.

In [ ]:
drinks['country_initial'] = drinks.country.apply(lambda x: x[0])

drinks.head(20)

Sometimes, we want to apply a min/max function to a set of columns and find which column produces the result. This can be achieved using the `idxmax` function.

In [ ]:
drinks.loc[:, 'beer_servings':'wine_servings']

In [ ]:
drinks.loc[:, 'beer_servings':'wine_servings'].apply(max, axis=1)

In [ ]:
drinks.loc[:, 'beer_servings':'wine_servings'].idxmax(axis=1)

## Exercise

1. Create a new column which contains the age each of the passengers would have had today (Titanic sunk in 1912)
2. Create a new column with the string value *survived* or *died* for each passenger
3. Create a new column `Deck` containig the symbol of the deck on which the passenger was travelling (the first letter of the cabin number)

## Joining data frames

There are multiple methods to join data frames, but we will focus on only two methods and forget about the rest:
- `pd.concat`: joins data frames vertically or horizontally
- `pd.merge`: performs database-like inner, outer, left, and right-joins based on an index or a column

In [ ]:
cities = pd.DataFrame({
    'country': ['Germany', 'Germany', 'Poland', 'Poland', 'Russia', 'Russia'],
    'city': ['Berlin', 'Munich', 'Warsaw', 'Cracow', 'Moscow', 'St Petersburg'],
    'is_capital': [True, False, True, False, True, False]
})

banks = pd.DataFrame({
    'country': ['Germany', 'Germany', 'Poland', 'France', 'France'],
    'name': ['Deutsche Bank', 'Commerzbank', 'Santander', 'Credit Agricole', 'BNP Paribas']
    
})

In [ ]:
pd.concat([cities, banks], axis=0)

In [ ]:
pd.concat([cities, banks], axis=1)

In [ ]:
pd.merge(cities, banks)

In [ ]:
banks.columns = ['country_name', 'bank_name']

pd.merge(cities, banks)

In [ ]:
pd.merge(cities, banks, left_on='country', right_on='country_name')

In [ ]:
cities.set_index('country', inplace=True)
banks.set_index('country_name', inplace=True)

pd.merge(cities, banks, left_index=True, right_index=True)

In [ ]:
pd.merge(cities, banks, left_index=True, right_index=True, how='inner')

In [ ]:
pd.merge(cities, banks, left_index=True, right_index=True, how='left')

In [ ]:
pd.merge(cities, banks, left_index=True, right_index=True, how='right')

In [ ]:
pd.merge(cities, banks, left_index=True, right_index=True, how='outer')

## Using pipes for data processing

The most pythonic way of performing a sequence of operations is to chain operators. However, this may not result in the most readable code. A simple library called `pipe` solves this problem by borrowing the chaining syntax from R.

Before observing `pipe` in action, let us first analyze the behavior of traditional `map()` and `filter()` functions.

In [ ]:
numbers = list(range(10))

even_numbers = list(filter(lambda x: x % 2 == 0, numbers))
even_numbers

In [ ]:
squares = list(map(lambda x: x**2, numbers))
squares

In [ ]:
def square(x): return x**2
def is_even(x): return x % 2 == 0

squares_of_even_numbers = list(map(square, filter(is_even, numbers)))
squares_of_even_numbers

The same functionality can be much easier achieved using pipes.

In [ ]:
from pipe import where, select

list(numbers 
     | where(is_even)
     | select(square)
    )

pipe operator `|` simply passes the output of one function as the input to another function. The remaining functions perform the following:
- `where`: filter out only those element of the iterable which fulfill the condition
- `select`: applies a function to each element of the iterable
- `traverse`: recursively unchain a sequence of iterables
- `groupby`: groups elements of an iterable 
- `dedup`: removes duplicates from an iterable

In [ ]:
from pipe import dedup

numbers = [1, 2, 3, 4, 5] * 3

print(f"Before deduplication: {numbers}")
print(f"After deduplication: {list(numbers | dedup)}")

In [ ]:
from pipe import traverse

nested_numbers = [1, 2, 3, [4, 5], [6, 7], 8, [9, 0]]

print(f"Unnested numbers: {list(nested_numbers | traverse)}")

In [ ]:
from pipe import groupby

numbers = list(range(10))

even_odd_numbers = list(
    numbers 
    | groupby(lambda x: "even" if x % 2 == 0 else "odd") 
    | select(lambda x: {x[0]: list(x[1])})
    )
    
print(f"Even and odd numbers: {even_odd_numbers}")

## Exercise

Using pipes, perform the following queries:

1. List unique ages of women who survived the sinking
2. Compute the mean age of passengers based on the port of embarkment
3. Create a list of titles (Mr., Mrs., etc.) and last names of passengers who died.